In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Carregando os dados
df_breastcancer = pd.read_csv("breastcancer.csv")

# Dividindo os dados em características (X) e rótulo (y)
X = df_breastcancer.iloc[:, :-1].values  # 30 primeiras colunas são características
y = df_breastcancer.iloc[:, -1].values.reshape(-1, 1)  # Última coluna é a saída

# Normalizando os dados de entrada
X_normalized = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))


In [ ]:
# Calculando média e desvio padrão
def mean_std(values):
    return np.mean(values), np.std(values)

# Função de ativação sigmoidal
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# Calculando as previsões
def predict(X, weights, bias):
    z = np.dot(X, weights) + bias
    return sigmoid(z)

## Gradient Descent

In [ ]:
# Implementando o modelo gd
def gradient_descent(X, y, weights, bias, learning_rate, iterations):
    n = float(len(X))

    for i in range(iterations):
        y_pred = predict(X, weights, bias)
        error = y - y_pred
        gradient_weights = -np.dot(X.T, error) / n
        gradient_bias = -np.mean(error)
        weights -= learning_rate * gradient_weights
        bias -= learning_rate * gradient_bias

    return weights, bias

## Stochastic Gradient Descent

In [ ]:
def stochastic_gradient_descent(X, y, weights, bias, learning_rate):
    n = float(len(X))
    for i in range(len(X)):
        x_i = X[i]
        y_i = y[i]
        y_pred = predict(x_i, weights, bias)
        error = y_i - y_pred
        gradient_weights = -x_i.reshape(-1,1) * error
        gradient_bias = -error
        weights -= learning_rate * gradient_weights
        bias -= learning_rate * gradient_bias
    return weights, bias


In [ ]:
# Função para Avaliação do Modelo
def evaluate_model(X_train, y_train, X_test, y_test, learning_rate, iterations, optimizer):
    # Inicializando os pesos e o viés
    weights = np.zeros(X_train.shape[1]).reshape(-1,1)
    bias = 0.0

    # Selecionando o algoritmo de otimização
    if optimizer == 'gd':
        weights, bias = gradient_descent(X_train, y_train, weights, bias, learning_rate, iterations)
    elif optimizer == 'sgd':
        weights, bias = stochastic_gradient_descent(X_train, y_train, weights, bias, learning_rate)

    # Fazendo previsões no conjunto de teste
    y_pred = predict(X_test, weights, bias)
    y_pred_binary = np.round(y_pred)

    # Calculando acurácia global
    accuracy = np.mean(y_pred_binary == y_test)

    # Calculando a acurácia por classe
    class_accuracy_0 = np.mean((y_pred_binary == y_test)[y_test == 0])
    class_accuracy_1 = np.mean((y_pred_binary == y_test)[y_test == 1])

    return accuracy, class_accuracy_0, class_accuracy_1

In [ ]:
# Definindo Hiperparâmetros
learning_rate = 0.1
iterations = 100
n_folds = 10
fold_size = len(X) // n_folds

# Avaliação do Modelo usando Gradient Descent
global_accuracy_list_gd = []
class_accuracy_0_list_gd = []
class_accuracy_1_list_gd = []

# Avaliação do Modelo usando Stochastic Gradient Descent
global_accuracy_list_sgd = []
class_accuracy_0_list_sgd = []
class_accuracy_1_list_sgd = []

for i in range(n_folds):
    start = i * fold_size
    end = (i + 1) * fold_size
    X_test = X_normalized[start:end]
    y_test = y[start:end]
    X_train = np.concatenate([X_normalized[:start], X_normalized[end:]])
    y_train = np.concatenate([y[:start], y[end:]])

    # Avaliando o modelo com Gradient Descent
    global_accuracy_gd, class_accuracy_0_gd, class_accuracy_1_gd = evaluate_model(X_train, y_train, X_test, y_test, learning_rate, iterations, optimizer='gd')

    # Armazenando as métricas de avaliação do GD
    global_accuracy_list_gd.append(global_accuracy_gd)
    class_accuracy_0_list_gd.append(class_accuracy_0_gd)
    class_accuracy_1_list_gd.append(class_accuracy_1_gd)

    # Avaliando o modelo com Stochastic Gradient Descent
    global_accuracy_sgd, class_accuracy_0_sgd, class_accuracy_1_sgd = evaluate_model(X_train, y_train, X_test, y_test, learning_rate, iterations, optimizer='sgd')

    # Armazenando as métricas de avaliação do SGD
    global_accuracy_list_sgd.append(global_accuracy_sgd)
    class_accuracy_0_list_sgd.append(class_accuracy_0_sgd)
    class_accuracy_1_list_sgd.append(class_accuracy_1_sgd)



In [ ]:
# Calculando as métricas de Gradient Descent
mean_global_accuracy_gd, std_global_accuracy_gd = np.mean(global_accuracy_list_gd), np.std(global_accuracy_list_gd)
mean_class_accuracy_0_gd, std_class_accuracy_0_gd = np.mean(class_accuracy_0_list_gd), np.std(class_accuracy_0_list_gd)
mean_class_accuracy_1_gd, std_class_accuracy_1_gd = np.mean(class_accuracy_1_list_gd), np.std(class_accuracy_1_list_gd)

# Imprimindo os resultados do Gradient Descent
print("Gradient Descent:")
print("Média da acurácia global:", mean_global_accuracy_gd)
print("Desvio padrão da acurácia global:", std_global_accuracy_gd)
print("Média da acurácia para classe 0:", mean_class_accuracy_0_gd)
print("Desvio padrão da acurácia para classe 0:", std_class_accuracy_0_gd)
print("Média da acurácia para classe 1:", mean_class_accuracy_1_gd)
print("Desvio padrão da acurácia para classe 1:", std_class_accuracy_1_gd)


In [ ]:
# Calculando as métricas de Stochastic Gradient Descent
mean_global_accuracy_sgd, std_global_accuracy_sgd = np.mean(global_accuracy_list_sgd), np.std(global_accuracy_list_sgd)
mean_class_accuracy_0_sgd, std_class_accuracy_0_sgd = np.mean(class_accuracy_0_list_sgd), np.std(class_accuracy_0_list_sgd)
mean_class_accuracy_1_sgd, std_class_accuracy_1_sgd = np.mean(class_accuracy_1_list_sgd), np.std(class_accuracy_1_list_sgd)

# Imprimindo os resultados do Stochastic Gradient Descent
print("Stochastic Gradient Descent:")
print("Média da acurácia global:", mean_global_accuracy_sgd)
print("Desvio padrão da acurácia global:", std_global_accuracy_sgd)
print("Média da acurácia para classe 0:", mean_class_accuracy_0_sgd)
print("Desvio padrão da acurácia para classe 0:", std_class_accuracy_0_sgd)
print("Média da acurácia para classe 1:", mean_class_accuracy_1_sgd)
print("Desvio padrão da acurácia para classe 1:", std_class_accuracy_1_sgd)